### （1）模型构建思路及调优过程：
#### a. 尝试了Unet、Attention Unet、SETR模型，最后发现Attention Unet精度可以达到最好精度0.51，SETR训练多个epoch后只有0.4+，Unet最低
#### b. batch_size依次尝试2、4、8、16... 根据显存能力尽量选取最高的batch_size
#### c. 训练集数据采用随机翻转进行增强

### （2）参考代码：
[10分钟上手PaddleSeg](https://aistudio.baidu.com/aistudio/projectdetail/1672610?channelType=0&channel=0)

### （3）代码内容说明如下
### 注：由于当初想把这个比赛方案作为公开项目baseline，所以提交结果后就没有保留模型参数，项目中暂存结果文件均为示例数据和模型

##  1 安装PaddleSeg及相关依赖

In [ ]:
#如果用户网络不佳，可以选择使用Gitee进行代码下载
! git clone https://gitee.com/paddlepaddle/PaddleSeg.git

Cloning into 'PaddleSeg'...
remote: Enumerating objects: 599, done.
remote: Counting objects: 100% (599/599), done.
remote: Compressing objects: 100% (403/403), done.
remote: Total 12383 (delta 251), reused 500 (delta 188), pack-reused 11784
Receiving objects: 100% (12383/12383), 267.47 MiB | 33.69 MiB/s, done.
Resolving deltas: 100% (8083/8083), done.
Checking connectivity... done.
Checking out files: 100% (1231/1231), done.


In [ ]:
%cd ~/PaddleSeg/
!pip install -r requirements.txt

/home/aistudio/PaddleSeg
Looking in indexes: https://mirror.baidu.com/pypi/simple/


## 2 训练自己的数据集

### 数据集配置说明

文件组织结构：
img_train为有标签数据，将其按照8:2划分为训练集和验证集
img_testA为测试数据

    remote_sensing
        |
        |--img_train
        |  |--xxx1.jpg
        |  |--xxx2.jpg
        |  |--...
        |
        |--lab_train
        |  |--xxx1.png
        |  |--xxx2.png
        |  |--...
        |
        |--train.txt
        |
        |--valid.txt
        |
        |--test.txt


In [ ]:
#解压数据至/data/remote_sensing文件夹下
!unzip -oq /home/aistudio/data/data80164/img_test.zip -d /home/aistudio/data/remote_sensing
!unzip -oq /home/aistudio/data/data80164/train_and_label.zip -d /home/aistudio/data/remote_sensing

In [ ]:
%cd ~/data/remote_sensing
# 生成 train.txt、valid.txt和test.txt列表文件
import os
import glob
imgs = os.listdir('/home/aistudio/data/remote_sensing/img_train')
with open("all_list.txt","w") as f:
    for img in imgs:
        img = os.path.join('img_train',img)
        label = img.replace('img','lab').replace('jpg','png')
        content = img + ' ' + label + '\n'
        f.write(content)

# 训练集、验证集比例分别约80%、20%。
!head -n 13331 all_list.txt > valid.txt #13331为验证集图片数量
!tail -n 53322 all_list.txt > train.txt #53322为训练集片数量

/home/aistudio/data/remote_sensing


## 3 模型训练与预测

### 配置文件中的数据集参数,同时把batch_size改成8

```
batch_size:8

train_dataset: 
  
  type: Dataset
  dataset_root: /home/aistudio/data/remote_sensing
  train_path: /home/aistudio/data/remote_sensing/train.txt
  num_classes: 4
  transforms:
    - type: Resize
      target_size: [512, 512]
    - type: RandomHorizontalFlip
    - type: Normalize
  mode: train

val_dataset: 
  type: Dataset
  dataset_root: /home/aistudio/data/remote_sensing
  val_path: /home/aistudio/data/remote_sensing/valid.txt
  num_classes: 4
  transforms:
    - type: Resize
      target_size: [512, 512]
    - type: Normalize
  mode: val
```

In [23]:
%cd ~/PaddleSeg/
!python train.py \
       --config configs/attention_unet/att_unet_remote_sensing.yml \
       --do_eval \
       --use_vdl \
       --save_interval 2000 \#大约一个epoch验证并保存3-4次模型
       --save_dir output

/home/aistudio/PaddleSeg
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-08-05 16:17:00 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICE

In [24]:
#模型推理并保存结果
!python predict.py \
       --config configs/attention_unet/att_unet_remote_sensing.yml \
       --model_path output/best_model/model.pdparams \
       --image_path /home/aistudio/data/remote_sensing/img_testA \
       --save_dir output/result

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
2021-08-05 16:18:01 [INFO]	
---------------Config Information---------------
batch_size: 8
iters: 80000
loss:
  coef:
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0.0
  learning_rate: 0.05
  power: 0.9
  type: PolynomialDecay
model:
  pretrained: null
  type: AttentionUNet
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-